In [ ]:
%pip install bs4 selenium webdriver_manager python-dotenv haversine pandas

In [11]:
from bs4 import BeautifulSoup
from navigator import KakaoRouteFinder
from IPython.display import HTML
import pandas as pd
import navigator
import coordinate
import os

os.environ['WDM_LOG_LEVEL'] = '0'

org, des = ('삼성고', '서울신림초')

org_coord = coordinate.get_coordinate(org)
des_coord = coordinate.get_coordinate(des)

(org_x, org_y) = coordinate.transform(org_coord[::-1], 'WGS84', 'WCONGNAMUL')
(des_x, des_y) = coordinate.transform(des_coord[::-1], 'WGS84', 'WCONGNAMUL')

print(f'https://map.kakao.com/?map_type=TYPE_MAP&target=car&rt={int(org_x)},{int(org_y)},{int(des_x)},{int(des_y)}&rt1={org}&rt2={des}')

routes = {}
routes['transit'] = KakaoRouteFinder().find_route_by_keyword('transit', org, des, time_delta=0.5, init_time=2.0)
navigator.extract_route('transit', routes['transit'])

https://map.kakao.com/?map_type=TYPE_MAP&target=car&rt=487553,1102737,483987,1104916&rt1=삼성고&rt2=서울신림초


[{'time': '26분', 'info': '도보17분 | 환승없음 | 요금 1,200원 | 3.3km'},
 {'time': '22분', 'info': '도보10분 | 환승1회 | 요금 1,200원 | 3.3km'},
 {'time': '27분', 'info': '도보13분 | 환승1회 | 요금 1,200원 | 3.8km'},
 {'time': '28분', 'info': '도보13분 | 환승1회 | 요금 1,200원 | 4.2km'},
 {'time': '34분', 'info': '도보20분 | 환승없음 | 요금 1,200원 | 3.4km'},
 {'time': '26분', 'info': '도보17분 | 환승1회 | 요금 1,200원 | 2.6km'},
 {'time': '34분', 'info': '도보16분 | 환승1회 | 요금 1,200원 | 4.4km'},
 {'time': '31분', 'info': '도보19분 | 환승1회 | 요금 1,200원 | 3.8km'},
 {'time': '34분', 'info': '도보19분 | 환승1회 | 요금 1,200원 | 4.3km'},
 {'time': '31분', 'info': '도보23분 | 환승없음 | 요금 2,150원 | 3.1km'}]

In [12]:
from tqdm import tqdm

soup = BeautifulSoup(routes['transit'], 'html.parser')
S = set()
for route in (soup.find_all('li', {"class": "TransitRouteItem"})):
    nodes = [name.text.strip().replace(" 승차", "").replace(" 하차", "").replace(" 환승", "").replace(" 정류장", "") for name in route.find_all('a', {"data-id": "name"})]
    # print(nodes)
    S.update(nodes)
    nodes = [node.text.strip() for node in route.find_all('li', {"class": "nodeName"})]
    # print(nodes)
    S.update(nodes)

seoul_bike = {}

for lat, lng in [org_coord, des_coord]:
    try:
        _, bike = coordinate.get_nearest_bike(float(lat), float(lng))
        bike_id, bike_name, bike_lat, bike_lng = bike[['대여소\n번호', '보관소(대여소)명', '위도(Y)', '경도(X)']]
        congnamul = coordinate.transform((bike_lng, bike_lat), 'WGS84', 'WCONGNAMUL')
        seoul_bike[bike_id] = ( \
            bike_name, \
            bike_lat, \
            bike_lng, \
            coordinate.get_distance(org_coord, (bike_lat, bike_lng)), \
            coordinate.get_distance(des_coord, (bike_lat, bike_lng)), \
            congnamul,
        )
    except:
        pass

for s in tqdm(S, desc="Retrieve Seoul Bike Stations: "):
    bus = coordinate.df_bus[coordinate.df_bus['정류소명'] == s]
    subway = coordinate.df_subway[coordinate.df_subway['name'] == s.replace('역', '')]
    try:
        if len(subway):
            lat, lng = subway[['lat(y)', 'lng']].iloc[0]
        elif len(bus):
            lat, lng = bus[['Y좌표', 'X좌표']].iloc[0]
        else:
            # print(s, 'N/A')
            continue
        _, bike = coordinate.get_nearest_bike(float(lat), float(lng))
        bike_id, bike_name, bike_lat, bike_lng = bike[['대여소\n번호', '보관소(대여소)명', '위도(Y)', '경도(X)']]
        congnamul = coordinate.transform((bike_lng, bike_lat), 'WGS84', 'WCONGNAMUL')
        seoul_bike[bike_id] = ( \
            bike_name, \
            bike_lat, \
            bike_lng, \
            coordinate.get_distance(org_coord, (bike_lat, bike_lng)), \
            coordinate.get_distance(des_coord, (bike_lat, bike_lng)), \
            congnamul,
        )
    except:
        # print('N/A')
        pass

rows = []
for id in seoul_bike.keys():
    name, lat, lng, org_dist, des_dist, congnamul = seoul_bike[id]
    coord = (lat, lng)
    link = f'https://map.kakao.com/?map_type=TYPE_MAP&target=transit&rt=,,{congnamul[0]},{congnamul[1]}&rt2={name}'
    rows.append([f'<a href="{link}">{name}</a>', coord, org_dist-des_dist, congnamul])

df = pd.DataFrame(rows, columns=["name", "coord", "gap_dist", "congnamul"])
HTML(df.sort_values(by="gap_dist").to_html(escape=False))

Retrieve Seoul Bike Stations: 100%|██████████| 30/30 [00:01<00:00, 15.85it/s]


,name,coord,gap_dist,congnamul
7,서울대학교 정문,"(37.46681976, 126.9488068)",-1.669597,"(488678.0, 1102057.0)"
0,나들목공원,"(37.46940613, 126.9445114)",-1.650333,"(487728.0, 1102775.0)"
13,서울산업정보학교,"(37.4706955, 126.9421997)",-1.266181,"(487217.0, 1103133.0)"
3,대학동 고시촌,"(37.47046661, 126.938652)",-0.758145,"(486433.0, 1103070.0)"
4,영락고등학교,"(37.47662735, 126.9436417)",-0.599540,"(487537.0, 1104779.0)"
5,서울신성초등학교,"(37.4711647, 126.9370194)",-0.435582,"(486072.0, 1103264.0)"
12,청량차도 육교 밑,"(37.58659363, 127.0469971)",-0.185168,"(510377.0, 1135290.0)"
9,동방1교,"(37.47182846, 126.9339218)",0.099103,"(485387.0, 1103449.0)"
10,도림천 신화교,"(37.4754715, 126.9364548)",0.137979,"(485948.0, 1104459.0)"
6,미림여고입구 교차로,"(37.47229385, 126.9334106)",0.227351,"(485274.0, 1103578.0)"


In [13]:
finder = KakaoRouteFinder()
bikes = sorted(seoul_bike.items(), key=lambda item: item[1][3]-item[1][4])
bike_routes = {}
for i, srt in tqdm(enumerate(bikes), desc="Finding Bike Routes: "):
    (_, (srt_name, srt_lat, srt_lng, _, _, srt_coord)) = srt
    for j in tqdm(range(i+1, len(bikes))):
        end = bikes[j]
        (_, (end_name, end_lat, end_lng, _, _, end_coord)) = end
        route = finder.find_route_by_congnamul('bike', srt_coord, end_coord, rt1=str(i), rt2=str(j), verbose=False, init=False, time_delta=0.2)
        route_id = f'{i}-{j}'
        bike_routes[route_id] = navigator.extract_route('bike', route)
del finder

Exception ignored in: <function KakaoRouteFinder.__del__ at 0x000001CB228383A0>
Traceback (most recent call last):
  File "c:\Users\lucet\Documents\22s-spatial-information-and-visualization\Project\navigator.py", line 87, in __del__
    self.driver.close()
  File "c:\Users\lucet\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 943, in close
    self.execute(Command.CLOSE)
  File "c:\Users\lucet\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 430, in execute
    self.error_handler.check_response(response)
  File "c:\Users\lucet\anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 247, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.WebDriverException: Message: chrome not reachable
  (Session info: chrome=102.0.5005.61)
Stacktrace:
Backtrace:
	Ordinal0 [0x0102D953+2414931]
	Ordinal0 [0x00FBF5E1+1963489]
	Ordinal0 [0x00EAC580+836992]
	Ordinal0 [0x00EA01E2+786914]


In [14]:
finder = KakaoRouteFinder()

org_routes = []
des_routes = []

for i, bike in tqdm(enumerate(bikes), desc="Finding Transit Routes"):
    (_, (name, _, _, _, _, coord)) = bike
    try:
        route = finder.find_route_by_congnamul('transit', (org_x, org_y), coord, rt1=org, rt2=str(i)+name, verbose=False, init=True, time_delta=0.5, init_time=2)
        org_routes.append(navigator.extract_route('transit', route))
    except:
        route = finder.find_route_by_congnamul('walk', (org_x, org_y), coord, rt1=org, rt2=str(i)+name, verbose=False, init=False, time_delta=0.5)
        org_routes.append(navigator.extract_route('walk', route))
        
    try:
        route = finder.find_route_by_congnamul('transit', coord, (des_x, des_y), rt1=str(i)+name, rt2=des, verbose=False, init=True, time_delta=0.5, init_time=2)
        des_routes.append(navigator.extract_route('transit', route))
    except:
        route = finder.find_route_by_congnamul('walk', coord, (des_x, des_y), rt1=str(i)+name, rt2=des, verbose=False, init=False, time_delta=0.5)
        des_routes.append(navigator.extract_route('walk', route))

del finder

Finding Transit Routes: 14it [01:31,  6.50s/it]


In [15]:
import pickle
filename = f'routes/{org}-{des}.pkl'

with open(filename, 'wb') as f:
    data = {'org_routes': org_routes, "des_routes": des_routes, "bikes": bikes, "bike_routes": bike_routes }
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)